# ⤵️Import Libraries

In [4]:
import os
import PyPDF2
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns

# 📂Load Data

In [5]:
os.chdir('..')
curr_dir = (os.getcwd()).replace('\\', '/')
file_path = curr_dir + '/Dataset/Project plan outline.pdf'

In [6]:
pdfFObj = open(file_path, 'rb')
pdfFile = PyPDF2.PdfReader(pdfFObj)

In [10]:
print(pdfFile.pages[0].extract_text())

ELEC6259  Project Preparation – Project Plan Template  1 
Title  Personal Loan defaulter prediction  
Student  name : Ashish Sasanapuri  
Supervisor name:  Jize Yan  
 
Aims/research question and Objectives  
Banking industry are one of the most important aspects of a nation’s economy. It  has in many ways helped in 
contributing to the economic household of the citizens. Among those is the loan business which has been 
established by the banks for the consumers to run their households. The loan lending system is a risky business 
that has to be carried out by the banks in order to balance their revenue. However, many banks follow a 
rigorous procedure of analyzing and running legal background checks on the customer to lend a loan. 
Customers borrow loans for various reasons, be it for a business , education, or house. The loan lending process 
can be a part of the traditional bank industry as well as the internet finance industry. The major problem faced 
by banks that affect their ec